In [31]:
import pandas as pd
agw = pd.read_stata("agwfile1")
gez = pd.read_stata("gezfile1")

S1 persoonlijke leningen
S2 doorlopende kredieten
S3 nog niet eerder genoemde kredieten
S4 financieringskredieten
S5 leningen bij familie of vrienden
S6 studieleningen
S7 kredieten op creditcards (1=ja, 0=nee)
S8 leningen die nog niet eerder genoemd zijn

### Eerst maak ik de Net-worth variabele

Dit doe ik dmv een functie die alle bezittingen (b1b, b2b, ..., b30b) bij elkaar optelt en daar de schulden (s1b, s2b, ..., s8b, x1b) vanaf trekt.

In [37]:
def calcWealth(row):
    wealth = (row["b1b"] + row["b2b"] + row["b3b"] + row["b4b"] + row["b6b"] + row["b7b"] + row["b8b"] + row["b11b"]
              + row["b12b"] + row["b13b"] + row["b14b"] + row["b15b"] + row["b16b"] + row["b17b"] + row["b18b"] + 
             row["b19ogb"] + row["b19hyb"] + row["b19vzb"] + row["b20b"] + row["b21b"] + row["b22b"] + row["b23b"] +
             row["b24b"] + row["b25b"] + row["b28b"] + row["b29b"] + row["b30b"])
    return wealth

In [36]:
def calcDebt(row):
    debt = (row["s1b"] + row["s2b"] + row["s3b"] + row["s4b"] + row["s5b"] + row["s6b"] + row["s7b"] + row["s8b"])
    
    return debt

In [ ]:
def netWorth(row):
    netWorth = row["wealth"] - row["debt"]
    return networth

In [33]:
test = agw

In [45]:
test["debt"] = test.apply(calcDebt, axis = 1)
x = 0
if (test.debt>0).bool():
    x += 1

print(x)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [34]:
test["wealth"] = test.apply(calcNetworth, axis = 1)
print(test.wealth)

0        91682.000000
1        10775.000000
2         1350.000000
3        23071.000000
4        39303.429688
5        25716.500000
6        54859.997803
7       426962.000000
8        24619.000000
9         6313.500000
10        2050.000000
11       17847.000000
12       64874.999878
13       41108.000000
14       55025.999878
15       44169.749512
16       19426.000000
17      380665.145996
18      487129.000000
19       21329.000000
20       32181.998047
21        8449.999756
22        2950.000000
23          10.000000
24        4973.000000
25      591599.102539
26           0.000000
27       93804.000000
28       48062.000000
29       18729.500000
            ...      
2329     15000.000000
2330     35851.000000
2331         0.000000
2332      6012.500000
2333    128726.000000
2334    143957.867188
2335     13170.661621
2336      3585.350296
2337      6000.000000
2338     42384.888550
2339      6250.000000
2340      8875.000000
2341     31095.686981
2342      7500.000000
2343      